In [ ]:
def display(*args, **kargs): pass

# Sampling
 
This lab demonstrates how to perform sampling including stratified sampling.  There are examples using both `DataFrame` and `RDD` operations

In [ ]:
baseDir = '/mnt/ml-class/'
irisTwoFeatures = sqlContext.read.parquet(baseDir + 'irisTwoFeatures.parquet')

In [ ]:
display(irisTwoFeatures)

When using a `DataFrame` we can call `.sampleBy` to return a stratified sample without using replacement.  `sampleBy` takes in a column and fractions for what percentage of each value to sample.  An explanation of `sampleBy` can be found under [DataFrame](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.sampleBy) for the Python API and under [DataFrameStatFunctions](http://spark.apache.org/docs/latest/api/scala/#org.apache.spark.sql.DataFrameStatFunctions) for the Scala API.

In [ ]:
help(irisTwoFeatures.sampleBy)

In [ ]:
stratifiedSample = irisTwoFeatures.sampleBy('label', {0: .10, 1: .20, 2: .30})
display(stratifiedSample)

How many?  And which labels did we sample?

In [ ]:
print 'total count: {0}'.format(stratifiedSample.count())

In [ ]:
labelCounts = (stratifiedSample
               .groupBy('label')
               .count()
               .orderBy('label'))
display(labelCounts)

Now let's sample with replacement from the `DataFrame`.

In [ ]:
help(irisTwoFeatures.sample)

In [ ]:
sampleWithReplace = irisTwoFeatures.sample(True, .20)
labelCountsReplace = (sampleWithReplace
                      .groupBy('label')
                      .count()
                      .orderBy('label'))
display(labelCountsReplace)

#### Convert to an RDD and sample from an RDD

First, we'll convert our `DataFrame` to an `RDD`.

In [ ]:
irisTwoFeaturesRDD = (irisTwoFeatures
                      .rdd
                      .map(lambda r: (r[1], r[0])))

print '\n'.join(map(repr, irisTwoFeaturesRDD.take(2)))

Next, we'll perform stratified sampling.

In [ ]:
help(irisTwoFeaturesRDD.sampleByKey)

In [ ]:
irisSampleRDD = irisTwoFeaturesRDD.sampleByKey(True, {0: 0.5, 1: 0.5, 2: 0.1}, seed=1)

print '\n'.join(map(repr, irisSampleRDD.take(5)))

What do our counts look like?

In [ ]:
print irisTwoFeaturesRDD.countByKey()
print irisSampleRDD.countByKey()

We could also call `sample` to perform a random sample instead of a stratified sample.